<a href="https://colab.research.google.com/github/Sandra1875/-03MIAR-Algoritmos-de-optimizaci-n/blob/main/AG3/Algoritmos_Sandra_Navarrete_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AG3- Actividad Guiada 3<br>
Nombre: Sandra Navarrete

https://github.com/raul27868/03MAIR---Algoritmos-de-Optimizacion/tree/master/AG3

https://colab.research.google.com/drive/1__qvcQlU5iIvi-oKFXxigj3qc-4ChZEa?usp=sharing


In [2]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

In [3]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion : 
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file) 

#Coordendas 51-city problem (Christofides/Eilon)
#file = "eil51.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/eil51.tsp", file)

#Coordenadas - 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/att48.tsp", file)




('swiss42.tsp', <http.client.HTTPMessage at 0x7fd2ab361550>)

In [4]:
#Modulos extras, no esenciales
import numpy as np
import matplotlib.pyplot as plt
import imageio                    #Para construir las imagenes con gif
from google.colab import files    #Para descargar ficheros generados con google colab

from tempfile import mkstemp      #Para genera carpetas y ficheros temporales

import random                     #Para generar valores aleatorios


In [5]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())



In [6]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
#dir(problem)

15

In [7]:
 
#Funcionas basicas
###############################################################################      
     
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion 

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)


 

In [8]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  Nodos = list(problem.get_nodes())
  
  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto
  
  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)
    
    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia
      
      
  print("Mejor solución:" , mejor_solucion) 
  print("Distancia     :" , mejor_distancia) 
  return mejor_solucion  

    
#Busqueda aleatoria con 5000 iteraciones    
solucion = busqueda_aleatoria(problem, 5000)    
 

Mejor solución: [0, 38, 30, 27, 17, 36, 11, 12, 41, 21, 24, 28, 16, 5, 39, 9, 10, 7, 18, 23, 25, 3, 32, 19, 14, 15, 35, 33, 20, 34, 40, 29, 22, 37, 31, 26, 2, 8, 13, 1, 6, 4]
Distancia     : 3467


In [9]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################

def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):
      
      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]  

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)         

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))
 

nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))

 

Distancia Solucion Incial: 3467
Distancia Mejor Solucion Local: 3283


In [10]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []
  
  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)
 
    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia 
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion) 
      print("Distancia     :" , mejor_distancia) 
      return mejor_solucion
      
    solucion_referencia = vecina
 
 
sol = busqueda_local(problem )  


En la iteracion  52 , la mejor solución encontrada es: [0, 7, 37, 15, 16, 14, 19, 13, 5, 4, 9, 21, 40, 24, 39, 32, 31, 35, 36, 17, 1, 6, 26, 18, 12, 11, 25, 10, 41, 23, 8, 29, 20, 33, 34, 38, 22, 30, 28, 27, 2, 3]
Distancia     : 1674


Para mejorar la nota búsqueda en entornos **variables**

***Búsquedas por Entornos Variables Anidados***

In [15]:
#Busqueda en entornos variables anidados:
def busqueda_entornosvariablesanidados(problem, N, M):
  mejor_solucion = []
  
  #Generar una solucion inicial de referencia(aleatoria)
  mejor_solucion = crear_solucion(Nodos)
  mejor_distancia = distancia_total(mejor_solucion, problem)
  #Repetir, hasta que se cumpla la condición de parada
  for k in range(N):                                #Criterio de parada: repetir N veces 
      #Generar al azar una solución x' del k-ésimo entorno de x.
      solucion_k = crear_solucion(Nodos)                #Genera una solucion aleatoria
      distancia_k = distancia_total(solucion_k, problem)  #Calcula el valor objetivo(distancia total)
      #Repetir, hasta que j = jmax, los pasos
      for j in range(M):
        solucion_j = crear_solucion(Nodos)                #Genera una solucion aleatoria
        distancia_j = distancia_total(solucion_j, problem)  #Calcula el valor objetivo(distancia total)
        #Exploración del entorno. Encontrar la mejor solución
        #Si f(x'') < f(x'), hacer x' ← x'' y j ← 1
        if distancia_k < distancia_j:                 #Compara con la mejor solucion k con la solucion j
          solucion_k = solucion_j
          distancia_k = distancia_j
          break
      if distancia_k < mejor_distancia:
        mejor_solucion = solucion_k
        mejor_distancia = distancia_k
        
  print("Mejor solución:" , mejor_solucion) 
  print("Distancia     :" , mejor_distancia) 
  return mejor_solucion  
  
solucion = busqueda_entornosvariablesanidados(problem, 5000, 100)    
 


Mejor solución: [0, 5, 32, 41, 8, 4, 36, 7, 1, 31, 17, 16, 3, 38, 24, 39, 18, 29, 9, 14, 35, 34, 33, 19, 27, 37, 30, 2, 15, 23, 21, 25, 13, 12, 10, 22, 6, 11, 28, 20, 40, 26]
Distancia     : 4115


**Reultado:** No fue posible la mejora con entornos variables aninados, el tiempo de procesamiento se incrementa, y la mejor distancia es superior a la encontrada en el metodo busqueda_local

In [16]:
###############################################################################
# SIMULATED ANNEALING
###############################################################################

#Generador de 1 solucion vecina 2-opt 100% aleatoria (intercambiar 2 nodos)
#Mejorable eligiendo otra forma de elegir una vecina.
def genera_vecina_aleatorio(solucion):

  #Se eligen dos nodos aleatoriamente
  i,j = sorted(random.sample( range(1,len(solucion)) , 2))
  
  #Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
  
 
genera_vecina_aleatorio(solucion) 


[0,
 5,
 32,
 41,
 8,
 4,
 36,
 7,
 1,
 31,
 17,
 16,
 3,
 38,
 24,
 39,
 18,
 29,
 9,
 14,
 35,
 34,
 33,
 19,
 27,
 37,
 30,
 2,
 15,
 23,
 21,
 25,
 13,
 6,
 10,
 22,
 12,
 11,
 28,
 20,
 40,
 26]

In [17]:

#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
  if random.random() <  math.exp( -1*d / T)  :
    return True
  else:
    return False

#Funcion de descenso de temperatura
def bajar_temperatura(T):
  return T*0.99
 



In [18]:
def recocido_simulado(problem, TEMPERATURA ):
  #problem = datos del problema
  #T = Temperatura
  
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  
  N=0
  while TEMPERATURA > .0001:
    N+=1
    #Genera una solución vecina
    vecina =genera_vecina_aleatorio(solucion_referencia)
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
      
    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
    #Si la nueva vecina es mejor se cambia  
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      #solucion_referencia = copy.deepcopy(vecina)
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)
 
  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

sol  = recocido_simulado(problem, 10000000)

 



La mejor solución encontrada es [0, 2, 28, 30, 29, 27, 4, 12, 10, 41, 40, 24, 21, 39, 9, 8, 18, 19, 14, 16, 11, 25, 23, 22, 38, 33, 20, 34, 32, 31, 35, 36, 17, 37, 15, 13, 5, 26, 6, 1, 7, 3]
con una distancia total de 1853


Para mejorar la nota **¿Se puede mejorar con otra elección no tan aleatoria**



In [37]:
import random

def genera_vecina_no_tan_aleatorio(solucion):
  #seleccionamos los nodos con una distribucion uniforme
  i= random.randint(1,len(solucion)-2) # el primero aleatorio
  j= i+1  #el segundo es su vecino mas cercano
  
  #Devuelve una nueva solución pero intercambiando los dos nodos elegidos
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
  
 
genera_vecina_no_tan_aleatorio(solucion) 


[0,
 5,
 41,
 32,
 8,
 4,
 36,
 7,
 1,
 31,
 17,
 16,
 3,
 38,
 24,
 39,
 18,
 29,
 9,
 14,
 35,
 34,
 33,
 19,
 27,
 37,
 30,
 2,
 15,
 23,
 21,
 25,
 13,
 12,
 10,
 22,
 6,
 11,
 28,
 20,
 40,
 26]

In [38]:
def recocido_simulado2(problem, TEMPERATURA ):
  #problem = datos del problema
  #T = Temperatura
  
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  
  N=0
  while TEMPERATURA > .0001:
    N+=1
    #Genera una solución vecina
    vecina =genera_vecina_no_tan_aleatorio(solucion_referencia)
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
      
    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
    #Si la nueva vecina es mejor se cambia  
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      #solucion_referencia = copy.deepcopy(vecina)
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)
 
  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

sol  = recocido_simulado2(problem, 10000000)


La mejor solución encontrada es [0, 18, 37, 34, 22, 8, 3, 14, 41, 30, 2, 7, 15, 26, 27, 28, 29, 33, 38, 40, 9, 25, 5, 36, 35, 12, 11, 21, 6, 32, 31, 1, 4, 39, 24, 23, 13, 19, 10, 20, 17, 16]
con una distancia total de 3758


No existió mejora al cambiar para que el segundo nodo sea el vecino mas cercado al primero nodo aleatorio